In [ ]:
import os
import time

from jax import numpy as np

import matplotlib
import matplotlib.animation 
import matplotlib.pyplot as plt
matplotlib.rcParams["animation.embed_limit"] = 512

import skimage
import skimage.io as sio
import skimage.transform
import fracatal

from fracatal.functional_jax import ft_convolve

import IPython

In [ ]:
"""
functions for building CA physics
"""

def make_gaussian(a, m, s):
    eps = 1e-9
    def gaussian(x):
        
        return a * np.exp(-((x-m)/(eps+s))**2 / 2)

    return gaussian
    
def make_mixed_gaussian(amplitudes, means, std_devs):
    
    def gaussian_mixture(x):
        results = 0.0 * x
        eps = 1e-9 # prevent division by 0
        
        for a, m, s in zip(amplitudes, means, std_devs):
            my_gaussian = make_gaussian(a, m, s)
            results += my_gaussian(x)
        
        return results

    return gaussian_mixture

def make_kernel_field(kernel_radius, dim=126):

    #dim = kernel_radius * 2 + 1
    

    x =  np.arange(-dim / 2, dim / 2 + 1, 1)
    xx, yy = np.meshgrid(x,x)

    rr = np.sqrt(xx**2 + yy**2) / kernel_radius

    return rr

def make_update_function(mean, standard_deviation):

    my_gaussian = make_gaussian(1.0, mean, standard_deviation)
    
    def lenia_update(x):
        """
        lenia update
        """
        return 2 * my_gaussian(x) - 1

    return lenia_update

def make_update_step(update_function, kernel, dt, clipping_function = lambda x: x, decimals=None):

    if decimals is not None:
        r = lambda x: np.round(x, decimals=decimals)
    else:
        r = lambda x: x
        
    def update_step(grid):

        
        neighborhoods = r(ft_convolve(r(grid), r(kernel)))
        dgrid_dt = r(update_function(neighborhoods))

        new_grid = r(clipping_function(r(grid) + dt * dgrid_dt))

        return new_grid

    return update_step

def make_make_kernel_function(amplitudes, means, standard_deviations, dim=126):
    
    def make_kernel(kernel_radius):
                
        gm = make_mixed_gaussian(amplitudes, means, standard_deviations)
        rr = make_kernel_field(kernel_radius, dim=dim)
        kernel = gm(rr)[None,None,:,:]
        kernel = kernel / kernel.sum()

        return kernel

    return make_kernel

# smooth life
def sigmoid_1(x, mu, alpha, gamma=1):
    return 1 / (1 + np.exp(-4 * (x - mu) / alpha))

def get_smooth_steps_fn(intervals, alpha=0.0125):
    """
    construct an update function from intervals.
    input intervals is a list of lists of interval bounds,
    each element contains the start and end of an interval. 

    # this code was adopated from rivesunder/yuca
    """
    
    def smooth_steps_fn(x):
        
        result = np.zeros_like(x)
        for bounds in intervals:
            result += sigmoid_1(x, bounds[0], alpha) * (1 - sigmoid_1(x, bounds[1], alpha))
        
        return result
    
    return smooth_steps_fn

def make_make_smoothlife_kernel_function(r_inner, r_outer):
    """
    r_inner and r_outer are 1/3. and 1.05 for Rafler's SmoothLife glider configuration
    """
    
    def make_smoothlife_kernel(kernel_radius=10):
        
        rr = make_kernel_field(kernel_radius)
        kernel = gm(rr)[None,None,:,:]
        kernel /= kernel.sum()

        return kernel
    
    return make_smoothlife_kernel
    

def make_smooth_interval(alpha=0.1470, intervals=[[0.2780, 0.3650]]):

    def smooth_interval(x):

        result = 0.0 * x

        for bounds in intervals:
            result += sigmoid_1(x, bounds[0], alpha) * (1-sigmoid_1(x, bounds[1], alpha))

        return 2 * result - 1

    return smooth_interval
                                                        
# smooth life
def make_make_smoothlife_kernel_function(r_inner, r_outer, dim=126):
    """
    r_inner and r_outer are 1/3. and 1.05 for Rafler's SmoothLife glider configuration
    """
    
    def make_smoothlife_kernel(kernel_radius=10):
        
        rr = make_kernel_field(kernel_radius, dim=dim)
        kernel = np.ones_like(rr)
        kernel = kernel.at[rr < r_inner].set(0.0)
        kernel = kernel.at[rr >= r_outer].set(0.0)
        
        kernel = (kernel / kernel.sum())[None,None,:,:]

        return kernel
    
    return make_smoothlife_kernel

def make_smooth_interval(alpha=0.1470, intervals=[[0.2780, 0.3650]]):

    def smooth_interval(x):

        result = 0.0 * x

        for bounds in intervals:
            result += sigmoid_1(x, bounds[0], alpha) * (1-sigmoid_1(x, bounds[1], alpha))

        return 2 * result - 1

    return smooth_interval
    
def make_smoothlife_update_function(intervals=[[0.2780, 0.3650]], \
                                    alpha=0.028):

    smooth = make_smooth_interval(alpha=alpha, intervals=intervals)
    
    def smoothlife_update(x):
        """
        smoothlife half-update
        """
        return 2 * smooth(x) - 1

    return smoothlife_update

def make_smoothlife_update_step(genesis_function, persistence_function, \
                                kernel, inner_kernel, dt, clipping_function = lambda x: x, \
                                decimals=None):

    if decimals is not None:
        r = lambda x: np.round(x, decimals=decimals)
    else:
        r = lambda x: x
        
    def update_step(grid):

        neighborhoods = r(ft_convolve(r(grid), r(kernel)))
        inner = r(ft_convolve(r(grid), r(inner_kernel)))

        genesis = r(genesis_function(neighborhoods))
        persistence = r(persistence_function(neighborhoods))
        dgrid_dt = (1-inner) * genesis + inner * persistence
                 
        new_grid = r(clipping_function(r(grid) + dt * dgrid_dt))

        return new_grid

    return update_step


In [ ]:
"""
animation functions
"""

def get_fig(grid):
    
    global subplot_0
    
    fig, ax = plt.subplots(1,1)
    
    subplot_0 = ax.imshow(grid.squeeze(), cmap="magma")
    
    return fig, ax

def update_frame(ii):
    
    global grid
    
    subplot_0.set_array(grid.squeeze())
    
    grid = update_step(grid)


In [ ]:
def v_stability_sweep(dts, krs, starting_grid, pattern, make_kernel, my_update, max_t, \
                    stride=4,\
                    persistence_update=None, \
                    make_inner_kernel=None, \
                    max_steps=1000, \
                    max_growth=2, \
                    min_growth=0.5, \
                    k0=31):

    clipping_fn = lambda x: np.clip(x, 0.0, 1.0)
    results = np.zeros((dts.shape[0], krs.shape[0],3))

    native_dim_h = pattern.shape[-2]
    native_dim_w = pattern.shape[-1]
    
    kernel = make_kernel(2)
    
    for jj, kr in enumerate(krs[::stride]):
        
        if make_inner_kernel is not None:
            inner_kernel = make_inner_kernel(k0)
            inner_kernel = np.zeros(1,kr.shape[0], inner_kernel.shape[-2], inner_kernel.shape[-1])
            
            for kk in range(kr.shape[0]):
                inner_kernel = kernel.at[:,kk,:,:].set(make_kernel(k0))
                
            update_step = make_smoothlife_update_step(my_update, persistence_update, \
                    kernel, inner_kernel, dt, \
                    clipping_fn)
        else:
            update_step = make_update_step(my_update, kernel, dt, clipping_fn)
            
        kernel = np.zeros(1, kr.shape[0],kernel.shape[-2], kernel.shape[-1])
        patterns = np.zeros(dts.shape[0], kr.shape[0],kernel.shape[-2], kernel.shape[-1])
        grid = np.zeros(dts.shape[0], kr.shape[0], starting_grid.shape[-2], starting_grid.shape[-1])
                        
        for kk in range(kr.shape[0]):
            kernel = kernel.at[:,kk,:,:].set(make_kernel(kr[kk]))
            
            scale_factor = kr[kk] / k0

            dim_h = int(native_dim_h * scale_factor)
            dim_w = int(native_dim_w * scale_factor)

            scaled_pattern = skimage.transform.resize(pattern, (1,1, dim_h, dim_w))
        
            grid = grid.at[:,:,:scaled_pattern.shape[-2], :scaled_pattern.shape[-1]].set(scaled_pattern)
    
            starting_sum = grid.sum(axis=(0,1), keepdims=True)
            
            explode = 0 * grid
            vanish = 0 * grid
            accumulated_t = 0 * grid
            total_steps = 0 * grid
            
        while accumulated_t < max_t and total_steps <= max_steps:

            grid = update_step(grid)
            
            g = grid.sum(axis=(0,1)) / starting_sum

            explode = g > max_growth
            vanish = g < min_growth
            
            accumulated_t += dt
            total_steps += 1
        
        if explode == True:
            results[ii,jj,0] = 1-accumulated_t / max_t
        elif vanish == True:
            results[ii,jj,2] = 1-accumulated_t / max_t
        else:
            results[ii,jj,1] = accumulated_t / max_t

    return results

    

In [ ]:
min_dt = 0.01
max_dt = 1.0
number_dt_steps = 2048
stride = 2
min_kr = 6
max_kr = 61
number_kr_steps = 2048

max_t = 2000
max_steps = 100
max_growth = 2.
min_growth = 0.5
k0 = 31
grid_dim = 128
starting_grid = np.zeros((1,1,grid_dim, grid_dim))

dts = np.arange(min_dt, max_dt, (max_dt-min_dt) / number_dt_steps)[:,None, None,None]
krs = np.arange(min_kr, max_kr, (max_kr-min_kr) / number_kr_steps)[:,None,None,None]

print(dts.shape, krs.shape)

clipping_fn = lambda x: np.clip(x, 0.0, 1.0)
results = np.zeros((dts.shape[0], krs.shape[0], 4))

native_dim_h = pattern.shape[-2]
native_dim_w = pattern.shape[-1]

kernel_dim = 125
make_kernel = make_make_kernel_function(amplitudes, means, standard_deviations, dim=kernel_dim)
kernel = make_kernel(2)

t0 = time.time()

for jj in range(krs[::stride].shape[0]):
    kr = krs[jj*stride:(jj+1)*stride]
        
    kernel = np.zeros((1, kr.shape[0],kernel.shape[-2], kernel.shape[-1]))
    patterns = np.zeros((dts.shape[0], kr.shape[0],kernel.shape[-2], kernel.shape[-1]))
    grid = np.zeros((dts.shape[0], kr.shape[0], starting_grid.shape[-2], starting_grid.shape[-1]))
                    
    for kk in range(kr.shape[0]):
        
    
        
        kernel = kernel.at[:,kk:kk+1,:,:].set(make_kernel(kr[kk].item()))
        scale_factor = kr[kk].item() / k0
        
        dim_h = int(native_dim_h * scale_factor)
        dim_w = int(native_dim_w * scale_factor)
        
        scaled_pattern = skimage.transform.resize(pattern, (1,1, dim_h, dim_w))
        
        grid = grid.at[:,kk:kk+1,:scaled_pattern.shape[-2], :scaled_pattern.shape[-1]].set(scaled_pattern)
        
    starting_sum = grid.sum(axis=(2,3), keepdims=True)
    
    explode = np.zeros((grid.shape[0], grid.shape[1],1,1))
    vanish = np.zeros((grid.shape[0], grid.shape[1],1,1))
    accumulated_t = np.zeros((grid.shape[0], grid.shape[1],1,1))
    total_steps = np.zeros((grid.shape[0], grid.shape[1],1,1))
    
    results_part = np.zeros((grid.shape[0], grid.shape[1], 4))
    
    update_step = make_update_step(my_update, kernel, dts, clipping_fn)
    
    red_cmap = plt.get_cmap("Reds")
    green_cmap = plt.get_cmap("Purples")
    blue_cmap = plt.get_cmap("Blues")
    
    while accumulated_t.min() < max_t and total_steps.max() <= max_steps:
    
        grid = update_step(grid)
        
        g = grid.sum(axis=(2,3), keepdims=True) / starting_sum
    
        # could be problems if some grid elements first explode, then vanish
        explode = explode + g > max_growth
        vanish = vanish + g < min_growth
        done = explode + vanish > 0
        
        accumulated_t += dts * (1 - done)
        total_steps += 1 * (1 - done)
    
    results_part = results_part.at[explode.squeeze() > 0].set(red_cmap(total_steps[explode.squeeze() > 0] / max_steps).squeeze())
    
    results_part = results_part.at[vanish.squeeze() > 0].set(blue_cmap(total_steps[vanish.squeeze() > 0] / max_steps).squeeze())
    
    results_part = results_part.at[done.squeeze() <= 0].set(green_cmap(total_steps[done.squeeze() <= 0] / max_steps).squeeze())
    
    results = results.at[:,jj*stride:(jj+1)*stride,:].set(results_part)

t1 = time.time()

In [ ]:
print(f"{(t1-t0)/60:.4f}")
print(results.shape)
fig, ax = plt.subplots(1,1, figsize=(20,20))
ax.imshow(results)
_ = ax.set_yticks(np.arange(0,dts.shape[0]))
_ = ax.set_yticklabels([f"{elem.item():.4f}" for elem in dts])
_ = ax.set_xticks(np.arange(0,krs.shape[0]))
_ = ax.set_xticklabels([f"{elem.item():.4f}" for elem in krs], rotation=90)

In [ ]:
print(f"{(t1-t0)/60:.4f}")
print(results.shape)
fig, ax = plt.subplots(1,1, figsize=(20,20))
ax.imshow(results)
_ = ax.set_yticks(np.arange(0,dts.shape[0]))
_ = ax.set_yticklabels([f"{elem.item():.4f}" for elem in dts])
_ = ax.set_xticks(np.arange(0,krs.shape[0]))
_ = ax.set_xticklabels([f"{elem.item():.4f}" for elem in krs], rotation=90)

In [ ]:

fig, ax = plt.subplots(1,1, figsize=(20,20))
ax.imshow(results)
_ = ax.set_yticks(np.arange(0,dts.shape[0]))
_ = ax.set_yticklabels([f"{elem.item():.4f}" for elem in dts])
_ = ax.set_xticks(np.arange(0,krs.shape[0]))
_ = ax.set_xticklabels([f"{elem.item():.4f}" for elem in krs], rotation=90)

In [ ]:
results.shape, results_part.shape, results_part.shape, results[:,jj*stride:(jj+1)*stride,:].shape

In [ ]:
results_part.shape,  explode.shape,  results_part[explode.squeeze() > 0].shape

In [ ]:




    while accumulated_t < max_t and total_steps <= max_steps:

        grid = update_step(grid)
        
        g = grid.sum(axis=(0,1)) / starting_sum

        explode = g > max_growth
        vanish = g < min_growth
        
        accumulated_t += dt
        total_steps += 1
    
    if explode == True:
        results[ii,jj,0] = 1-accumulated_t / max_t
    elif vanish == True:
        results[ii,jj,2] = 1-accumulated_t / max_t
    else:
        results[ii,jj,1] = accumulated_t / max_t

return results


In [ ]:
"""

     if make_inner_kernel is not None:
        inner_kernel = make_inner_kernel(k0)
        inner_kernel = np.zeros(1,kr.shape[0], inner_kernel.shape[-2], inner_kernel.shape[-1])
        
        for kk in range(kr.shape[0]):
            inner_kernel = kernel.at[:,kk,:,:].set(make_kernel(k0))
            
        update_step = make_smoothlife_update_step(my_update, persistence_update, \
                kernel, inner_kernel, dt, \
                clipping_fn)
    else:
        update_step = make_update_step(my_update, kernel, dt, clipping_fn)

"""

fig, ax = plt.subplots(2,2)
ax[0,0].imshow(grid[0,0,:,:].squeeze())
ax[0,1].imshow(grid[0,1,:,:].squeeze())
ax[1,0].imshow(grid[0,2,:,:].squeeze())
ax[1,1].imshow(grid[0,3,:,:].squeeze())
plt.show()

fig, ax = plt.subplots(2,2)
ax[0,0].imshow(kernel[0,0,:,:].squeeze())
ax[0,1].imshow(kernel[0,1,:,:].squeeze())
ax[1,0].imshow(kernel[0,2,:,:].squeeze())
ax[1,1].imshow(kernel[0,3,:,:].squeeze())
plt.show()

In [ ]:
def stability_sweep(dts, krs, starting_grid, pattern, make_kernel, my_update, max_t, \
                    persistence_update=None, \
                    make_inner_kernel=None, \
                    max_steps=1000, \
                    max_growth=2, \
                    min_growth=0.5, \
                    k0=31):


    clipping_fn = lambda x: np.clip(x, 0.0, 1.0)
    results = np.zeros((dts.shape[0], krs.shape[0],3))
    max_growth = 1.5
    min_growth = 0.5    


    native_dim_h = pattern.shape[-2]
    native_dim_w = pattern.shape[-1]
    
    for ii, dt in enumerate(dts):
        for jj, kr in enumerate(krs):
            
            kernel = make_kernel(kr)
            
            if make_inner_kernel is not None:
                inner_kernel = make_inner_kernel(k0)
                update_step = make_smoothlife_update_step(my_update, persistence_update, \
                        kernel, inner_kernel, dt, \
                        clipping_fn)
            else:
                update_step = make_update_step(my_update, kernel, dt, clipping_fn)

            accumulated_t = 0.0
            total_steps = 0
            scale_factor = kr / k0

            dim_h = int(native_dim_h * scale_factor)
            dim_w = int(native_dim_w * scale_factor)
                    
            scaled_pattern = skimage.transform.resize(pattern, (1,1, dim_h, dim_w))
            
            grid = starting_grid * 0.0
            grid[:,:,:scaled_pattern.shape[-2], :scaled_pattern.shape[-1]] = scaled_pattern

            starting_sum = grid.sum()
            
            explode = False
            vanish = False

            if(0):
                
                plt.figure()
                plt.imshow(scaled_pattern.squeeze())
                plt.show()
                
                plt.figure()
                plt.imshow(grid.squeeze())
                plt.show()
            
            while accumulated_t < max_t and total_steps <= max_steps:
 
                grid = update_step(grid)
                
                g = grid.sum() / starting_sum
                
                accumulated_t += dt
                total_steps += 1
                if g > max_growth:
                    explode = True
                    break
                if g < min_growth:
                    vanish = True
                    break
            if(0):
                plt.figure()
                plt.imshow(grid.squeeze())
                plt.show()
                
            if explode == True:
                results[ii,jj,0] = 1-accumulated_t / max_t
            elif vanish == True:
                results[ii,jj,2] = 1-accumulated_t / max_t
            else:
                results[ii,jj,1] = accumulated_t / max_t

    return results

    

# SmoothLife glider

In [ ]:

pattern_filepath = os.path.join("..", "patterns", "smoothlife_single.npy")

pattern = np.load(pattern_filepath)[None,None,:,:]
plt.figure()
plt.imshow(pattern.squeeze(), cmap="magma")
plt.title("pattern")
plt.show()

In [ ]:
num_frames = 100
kr = 10
k0 = 10
dt= 1.0
grid_dim = 132

k0 = 10
r_inner = 1/3.
r_outer = 1.05

gen = make_smooth_interval(alpha=0.028, intervals=[[0.2780, 0.3650]])
per = make_smooth_interval(alpha=0.1470, intervals=[[0.2670, 0.4450]])

make_sl_kernel = make_make_smoothlife_kernel_function(r_inner, r_outer,dim = 126)
make_sl_inner_kernel = make_make_smoothlife_kernel_function(0.0, r_inner, dim = 126)

inner_kernel = make_sl_inner_kernel(k0)
kernel = make_sl_kernel(kr)

update_step = make_smoothlife_update_step(gen, per, \
        kernel, inner_kernel, dt, \
        clipping_function = lambda x: np.clip(x, 0, 1.0), \
        decimals=None)


scale_factor = kr / k0
native_dim_h = pattern.shape[-2]
native_dim_w = pattern.shape[-1]
dim_h = int(native_dim_h * scale_factor)
dim_w = int(native_dim_w * scale_factor)
        
scaled_pattern = skimage.transform.resize(pattern, (1,1, dim_h, dim_w),order=5)

grid = np.zeros((1, 1, grid_dim, grid_dim))
grid = grid.at[:,:,:scaled_pattern.shape[-2], :scaled_pattern.shape[-1]].set(scaled_pattern)

fig, ax = get_fig(grid[0])
plt.show()

IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_frame, frames=num_frames, interval=10).to_jshtml())

In [ ]:
min_dt = 0.01
max_dt = 1.0 
number_dt_steps = 64
max_dt = max_dt + 2*(max_dt-min_dt) / number_dt_steps
stride = 2
min_kr = 6
max_kr = 41
number_kr_steps = 64

max_t = 10000
max_steps = 1000
max_growth = 2.
min_growth = 0.5
k0 = 10
grid_dim = 192
make_inner_kernel = None
per_update = None


starting_grid = np.zeros((1,1,grid_dim, grid_dim))

dts = np.arange(min_dt, max_dt, (max_dt-min_dt) / number_dt_steps)[:,None, None,None]
krs = np.arange(min_kr, max_kr, (max_kr-min_kr) / number_kr_steps)[:,None,None,None]

print(dts.shape, krs.shape)

clipping_fn = lambda x: np.clip(x, 0.0, 1.0)
results = np.zeros((dts.shape[0], krs.shape[0], 4))

native_dim_h = pattern.shape[-2]
native_dim_w = pattern.shape[-1]

kernel_dim = 125
#make_kernel = make_make_kernel_function(amplitudes, means, standard_deviations, dim=kernel_dim)

make_kernel = make_make_smoothlife_kernel_function(r_inner, r_outer,dim = 126)
#make_inner_kernel = make_make_smoothlife_kernel_function(0.0, r_inner, dim = 126)


kernel = make_kernel(2)

t0 = time.time()

#my_update = gen
#per_update = per

for jj in range(krs[::stride].shape[0]):
    kr = krs[jj*stride:(jj+1)*stride]
        
    kernel = np.zeros((1, kr.shape[0],kernel.shape[-2], kernel.shape[-1]))
    inner_kernel = np.zeros((1, kr.shape[0],kernel.shape[-2], kernel.shape[-1]))
    patterns = np.zeros((dts.shape[0], kr.shape[0],kernel.shape[-2], kernel.shape[-1]))
    grid = np.zeros((dts.shape[0], kr.shape[0], starting_grid.shape[-2], starting_grid.shape[-1]))
                    
    for kk in range(kr.shape[0]):
        
        kernel = kernel.at[:,kk:kk+1,:,:].set(make_kernel(kr[kk].item()))
        scale_factor = kr[kk].item() / k0
        
        dim_h = int(native_dim_h * scale_factor)
        dim_w = int(native_dim_w * scale_factor)
        
        scaled_pattern = skimage.transform.resize(pattern, (1,1, dim_h, dim_w))
        
        grid = grid.at[:,kk:kk+1,:scaled_pattern.shape[-2], :scaled_pattern.shape[-1]].set(scaled_pattern)

    
    if make_inner_kernel is not None:
        inner_kernel = inner_kernel.at[:,:,:,:].set(make_inner_kernel(k0))
        
    starting_sum = grid.sum(axis=(2,3), keepdims=True)
    
    explode = np.zeros((grid.shape[0], grid.shape[1],1,1))
    vanish = np.zeros((grid.shape[0], grid.shape[1],1,1))
    accumulated_t = np.zeros((grid.shape[0], grid.shape[1],1,1))
    total_steps = np.zeros((grid.shape[0], grid.shape[1],1,1))
    
    results_part = np.zeros((grid.shape[0], grid.shape[1], 4))

    if make_inner_kernel is not None:
        update_step = make_smoothlife_update_step(gen, per, \
                kernel, inner_kernel, dts, \
                clipping_function = lambda x: np.clip(x, 0, 1.0), \
                decimals=None)

    else:
        update_step = make_update_step(my_update, kernel, dts, clipping_fn)

    
    
    red_cmap = plt.get_cmap("Reds")
    green_cmap = plt.get_cmap("Purples")
    blue_cmap = plt.get_cmap("Blues")
    total_steps_counter = 0
    
    while accumulated_t.min() < max_t and total_steps_counter <= max_steps:
    
        grid = update_step(grid)
        
        g = grid.sum(axis=(2,3), keepdims=True) / starting_sum
    
        explode = explode + (g > max_growth)
        vanish = vanish + (g < min_growth)
        done = (explode + vanish) > 0
        
        accumulated_t += dts * (1 - done)
        total_steps += 1 * (1 - done)
        total_steps_counter += 1
    
    results_part = results_part.at[done.squeeze() <= 0].set(green_cmap(accumulated_t[done.squeeze() <= 0] / max_t).squeeze())
    
    results_part = results_part.at[explode.squeeze() > 0].set(red_cmap(accumulated_t[explode.squeeze() > 0] / max_t).squeeze())
    results_part = results_part.at[vanish.squeeze() > 0].set(blue_cmap(accumulated_t[vanish.squeeze() > 0] / max_t).squeeze())
    
    results_part = results_part.at[done.squeeze() <= 0].set(green_cmap(total_steps[done.squeeze() <= 0] / max_steps).squeeze())
    
    results_part = results_part.at[explode.squeeze() > 0].set(red_cmap(total_steps[explode.squeeze() > 0] / max_steps).squeeze())
    results_part = results_part.at[vanish.squeeze() > 0].set(blue_cmap(total_steps[vanish.squeeze() > 0] / max_steps).squeeze())
        
    results = results.at[:,jj*stride:(jj+1)*stride,:].set(results_part)

t1 = time.time()

print(f"{t1-t0:.3f}")
fig, ax = plt.subplots(1,1, figsize=(20,20))
ax.imshow(results)
_ = ax.set_yticks(np.arange(0,dts.shape[0]))
_ = ax.set_yticklabels([f"{elem.item():.4f}" for elem in dts])
_ = ax.set_xticks(np.arange(0,krs.shape[0]))
_ = ax.set_xticklabels([f"{elem.item():.4f}" for elem in krs], rotation=90)

In [ ]:
results.shape

In [ ]:
(16, 1, 1, 1) (16, 1, 1, 1)
70.119
(8, 1, 1, 1) (8, 1, 1, 1)
27.902
(32, 1, 1, 1) (32, 1, 1, 1)
212.910
(64, 1, 1, 1) (64, 1, 1, 1)
744.921

In [ ]:
min_dt = 0.01
max_dt = 1.0
number_dt_steps = 32
max_dt = max_dt + 2*(max_dt-min_dt) / number_dt_steps
stride = 2
min_kr = 5
max_kr = 45
number_kr_steps = 32

max_t = 10000
max_steps = 1000
max_growth = 2.
min_growth = 0.5
k0 = 10
grid_dim = 192
make_inner_kernel = None
per_update = None


starting_grid = np.zeros((1,1,grid_dim, grid_dim))

dts = np.arange(min_dt, max_dt, (max_dt-min_dt) / number_dt_steps)[:,None, None,None]
krs = np.arange(min_kr, max_kr, (max_kr-min_kr) / number_kr_steps)[:,None,None,None]

print(dts.shape, krs.shape)

clipping_fn = lambda x: np.clip(x, 0.0, 1.0)
results = np.zeros((dts.shape[0], krs.shape[0], 4))

native_dim_h = pattern.shape[-2]
native_dim_w = pattern.shape[-1]

kernel_dim = 125
#make_kernel = make_make_kernel_function(amplitudes, means, standard_deviations, dim=kernel_dim)

make_kernel = make_make_smoothlife_kernel_function(r_inner, r_outer,dim = 126)
#make_inner_kernel = make_make_smoothlife_kernel_function(0.0, r_inner, dim = 126)


kernel = make_kernel(2)

t0 = time.time()

#my_update = gen
#per_update = per

for jj in range(krs[::stride].shape[0]):
    kr = krs[jj*stride:(jj+1)*stride]
        
    kernel = np.zeros((1, kr.shape[0],kernel.shape[-2], kernel.shape[-1]))
    inner_kernel = np.zeros((1, kr.shape[0],kernel.shape[-2], kernel.shape[-1]))
    patterns = np.zeros((dts.shape[0], kr.shape[0],kernel.shape[-2], kernel.shape[-1]))
    grid = np.zeros((dts.shape[0], kr.shape[0], starting_grid.shape[-2], starting_grid.shape[-1]))
                    
    for kk in range(kr.shape[0]):
        
        kernel = kernel.at[:,kk:kk+1,:,:].set(make_kernel(kr[kk].item()))
        scale_factor = kr[kk].item() / k0
        
        dim_h = int(native_dim_h * scale_factor)
        dim_w = int(native_dim_w * scale_factor)
        
        scaled_pattern = skimage.transform.resize(pattern, (1,1, dim_h, dim_w))
        
        grid = grid.at[:,kk:kk+1,:scaled_pattern.shape[-2], :scaled_pattern.shape[-1]].set(scaled_pattern)

    
    if make_inner_kernel is not None:
        inner_kernel = inner_kernel.at[:,:,:,:].set(make_inner_kernel(k0))
        
    starting_sum = grid.sum(axis=(2,3), keepdims=True)
    
    explode = np.zeros((grid.shape[0], grid.shape[1],1,1))
    vanish = np.zeros((grid.shape[0], grid.shape[1],1,1))
    accumulated_t = np.zeros((grid.shape[0], grid.shape[1],1,1))
    total_steps = np.zeros((grid.shape[0], grid.shape[1],1,1))
    
    results_part = np.zeros((grid.shape[0], grid.shape[1], 4))

    if make_inner_kernel is not None:
        update_step = make_smoothlife_update_step(gen, per, \
                kernel, inner_kernel, dts, \
                clipping_function = lambda x: np.clip(x, 0, 1.0), \
                decimals=None)

    else:
        update_step = make_update_step(my_update, kernel, dts, clipping_fn)

    
    
    red_cmap = plt.get_cmap("Reds")
    green_cmap = plt.get_cmap("Purples")
    blue_cmap = plt.get_cmap("Blues")
    total_steps_counter = 0
    
    while accumulated_t.min() < max_t and total_steps_counter <= max_steps:
    
        grid = update_step(grid)
        
        g = grid.sum(axis=(2,3), keepdims=True) / starting_sum
    
        explode = explode + (g > max_growth)
        vanish = vanish + (g < min_growth)
        done = (explode + vanish) > 0
        
        accumulated_t += dts * (1 - done)
        total_steps += 1 * (1 - done)
        total_steps_counter += 1
    
    results_part = results_part.at[done.squeeze() <= 0].set(green_cmap(accumulated_t[done.squeeze() <= 0] / max_t).squeeze())
    
    results_part = results_part.at[explode.squeeze() > 0].set(red_cmap(accumulated_t[explode.squeeze() > 0] / max_t).squeeze())
    results_part = results_part.at[vanish.squeeze() > 0].set(blue_cmap(accumulated_t[vanish.squeeze() > 0] / max_t).squeeze())
    
    results_part = results_part.at[done.squeeze() <= 0].set(green_cmap(total_steps[done.squeeze() <= 0] / max_steps).squeeze())
    
    results_part = results_part.at[explode.squeeze() > 0].set(red_cmap(total_steps[explode.squeeze() > 0] / max_steps).squeeze())
    results_part = results_part.at[vanish.squeeze() > 0].set(blue_cmap(total_steps[vanish.squeeze() > 0] / max_steps).squeeze())
        
    results = results.at[:,jj*stride:(jj+1)*stride,:].set(results_part)

t1 = time.time()

print(f"{t1-t0:.3f}")
fig, ax = plt.subplots(1,1, figsize=(20,20))
ax.imshow(results)
_ = ax.set_yticks(np.arange(0,dts.shape[0]))
_ = ax.set_yticklabels([f"{elem.item():.4f}" for elem in dts])
_ = ax.set_xticks(np.arange(0,krs.shape[0]))
_ = ax.set_xticklabels([f"{elem.item():.4f}" for elem in krs], rotation=90)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(20,20))
ax.imshow(results)
_ = ax.set_yticks(np.arange(0,dts.shape[0]))
_ = ax.set_yticklabels([f"{elem.item():.4f}" for elem in dts])
_ = ax.set_xticks(np.arange(0,krs.shape[0]))
_ = ax.set_xticklabels([f"{elem.item():.4f}" for elem in krs], rotation=90)

In [ ]:
x = np.arange(0,1.0, 1/32) * np.ones((32,32))
plt.subplot(131)
plt.imshow(green_cmap(x))
plt.subplot(132)
plt.imshow(blue_cmap(x))
plt.subplot(133)
plt.imshow(red_cmap(x))

#plt.imshow(x); plt.colorbar()

In [ ]:
plt.subplot(131)
plt.imshow(green_cmap(x))"""
(32, 1, 1, 1) (32, 1, 1, 1)
177.665
(64, 1, 1, 1) (64, 1, 1, 1)
737.714
"""

In [ ]:
#grid = grid.at[:,:,:pattern.shape[-2], :pattern.shape[-1]].set(pattern)
        
offset = 32
for jj in range(offset,offset+8): #number_dt_steps):
    plt.figure()
    for ii in range(8):
        
        plt.subplot(3,3,ii+1)
        plt.imshow(grid[-jj,ii,:,:].squeeze())
        plt.title(f"{dts[-jj].item():.3f}, {krs[ii].item():.3f}")
    
    

In [ ]:
dts.shape

In [ ]:
min_dt = 0.01
max_dt = 1.0
number_dt_steps = 256
stride = 32
min_kr = 6
max_kr = 61
number_kr_steps = 256

max_t = 2000
max_steps = 100
max_growth = 2.
min_growth = 0.5
k0 = 31
grid_dim = 192
starting_grid = np.zeros((1,1,grid_dim, grid_dim))

dts = np.arange(min_dt, max_dt, (max_dt-min_dt) / number_dt_steps)[:,None, None,None]
krs = np.arange(min_kr, max_kr, (max_kr-min_kr) / number_kr_steps)[:,None,None,None]

print(dts.shape, krs.shape)

clipping_fn = lambda x: np.clip(x, 0.0, 1.0)
results = np.zeros((dts.shape[0], krs.shape[0], 4))

native_dim_h = pattern.shape[-2]
native_dim_w = pattern.shape[-1]

kernel_dim = 125
make_kernel = make_make_kernel_function(amplitudes, means, standard_deviations, dim=kernel_dim)
kernel = make_kernel(2)

t0 = time.time()

for jj in range(krs[::stride].shape[0]):
    kr = krs[jj*stride:(jj+1)*stride]
        
    kernel = np.zeros((1, kr.shape[0],kernel.shape[-2], kernel.shape[-1]))
    patterns = np.zeros((dts.shape[0], kr.shape[0],kernel.shape[-2], kernel.shape[-1]))
    grid = np.zeros((dts.shape[0], kr.shape[0], starting_grid.shape[-2], starting_grid.shape[-1]))
                    
    for kk in range(kr.shape[0]):
        
        kernel = kernel.at[:,kk:kk+1,:,:].set(make_kernel(kr[kk].item()))
        scale_factor = kr[kk].item() / k0
        
        dim_h = int(native_dim_h * scale_factor)
        dim_w = int(native_dim_w * scale_factor)
        
        scaled_pattern = skimage.transform.resize(pattern, (1,1, dim_h, dim_w))
        
        grid = grid.at[:,kk:kk+1,:scaled_pattern.shape[-2], :scaled_pattern.shape[-1]].set(scaled_pattern)
        
    starting_sum = grid.sum(axis=(2,3), keepdims=True)
    
    explode = np.zeros((grid.shape[0], grid.shape[1],1,1))
    vanish = np.zeros((grid.shape[0], grid.shape[1],1,1))
    accumulated_t = np.zeros((grid.shape[0], grid.shape[1],1,1))
    total_steps = np.zeros((grid.shape[0], grid.shape[1],1,1))
    
    results_part = np.zeros((grid.shape[0], grid.shape[1], 4))
    
    update_step = make_update_step(my_update, kernel, dts, clipping_fn)
    
    red_cmap = plt.get_cmap("Reds")
    green_cmap = plt.get_cmap("Purples")
    blue_cmap = plt.get_cmap("Blues")
    
    while accumulated_t.min() < max_t and total_steps.max() <= max_steps:
    
        grid = update_step(grid)
        
        g = grid.sum(axis=(2,3), keepdims=True) / starting_sum
    
        # could be problems if some grid elements first explode, then vanish
        explode = explode + g > max_growth
        vanish = vanish + g < min_growth
        done = explode + vanish > 0
        
        accumulated_t += dts * (1 - done)
        total_steps += 1 * (1 - done)
    
    results_part = results_part.at[explode.squeeze() > 0].set(red_cmap(total_steps[explode.squeeze() > 0] / max_steps).squeeze())
    
    results_part = results_part.at[vanish.squeeze() > 0].set(blue_cmap(total_steps[vanish.squeeze() > 0] / max_steps).squeeze())
    
    results_part = results_part.at[done.squeeze() <= 0].set(green_cmap(total_steps[done.squeeze() <= 0] / max_steps).squeeze())
    
    results = results.at[:,jj*stride:(jj+1)*stride,:].set(results_part)

t1 = time.time()

print(f"{t1-t0:.3f}")
fig, ax = plt.subplots(1,1, figsize=(20,20))
ax.imshow(results)
_ = ax.set_yticks(np.arange(0,dts.shape[0]))
_ = ax.set_yticklabels([f"{elem.item():.4f}" for elem in dts])
_ = ax.set_xticks(np.arange(0,krs.shape[0]))
_ = ax.set_xticklabels([f"{elem.item():.4f}" for elem in krs], rotation=90)

In [ ]:
min_dt = 0.01
max_dt = 1.0
number_dt_steps = 16
min_kr = 6
max_kr = 63
number_kr_steps = 16

dts = np.arange(min_dt, max_dt, (max_dt-min_dt) / number_dt_steps)
krs = np.arange(min_kr, max_kr, (max_kr-min_kr) / number_kr_steps)
max_t = dts.max() * 200

number_samples = 1
dim = 256
grid = np.zeros((number_samples,1,dim,dim))

make_outer_kernel = make_make_smoothlife_kernel_function(r_inner, r_outer, dim = 126)
make_inner_kernel = make_make_smoothlife_kernel_function(0.0, r_inner, dim = 126)
t0 = time.time()

results = stability_sweep(dts, krs, grid, pattern, make_outer_kernel, gen, max_t, per, make_inner_kernel, k0=10, max_steps=1000)
t1 = time.time()

print(f"elapsed: {t1-t0:.4f}")

fig, ax = plt.subplots(1,1, figsize=(20,20))
ax.imshow(results)
_ = ax.set_yticks(np.arange(0,dts.shape[0]))
_ = ax.set_yticklabels([f"{elem:.4f}" for elem in dts])
_ = ax.set_xticks(np.arange(0,krs.shape[0]))
_ = ax.set_xticklabels([f"{elem:.4f}" for elem in krs], rotation=90)

In [ ]:
min_dt = 0.15
max_dt = 0.3
number_dt_steps = 16
min_kr = 10
max_kr = 13
number_kr_steps = 16

dts = np.arange(min_dt, max_dt, (max_dt-min_dt) / number_dt_steps)
krs = np.arange(min_kr, max_kr, (max_kr-min_kr) / number_kr_steps)
max_t = dts.max() * 200

number_samples = 1
dim = 256
grid = np.zeros((number_samples,1,dim,dim))

make_outer_kernel = make_make_smoothlife_kernel_function(r_inner, r_outer, dim = 126)
make_inner_kernel = make_make_smoothlife_kernel_function(0.0, r_inner, dim = 126)
t0 = time.time()

results = stability_sweep(dts, krs, grid, pattern, make_outer_kernel, gen, max_t, per, make_inner_kernel, k0=10, max_steps=1000)
t1 = time.time()

print(f"elapsed: {t1-t0:.4f}")

fig, ax = plt.subplots(1,1, figsize=(20,20))
ax.imshow(results)
_ = ax.set_yticks(np.arange(0,dts.shape[0]))
_ = ax.set_yticklabels([f"{elem:.4f}" for elem in dts])
_ = ax.set_xticks(np.arange(0,krs.shape[0]))
_ = ax.set_xticklabels([f"{elem:.4f}" for elem in krs], rotation=90)

# _H. natans_

In [ ]:
# common setup for H. natans

# the neighborhood kernel
amplitudes = [0.5, 1.0, 0.6667]
means = [0.0938, 0.2814, 0.4690]
standard_deviations = [0.0330, 0.0330, 0.0330]
kernel_radius = 31

make_kernel = make_make_kernel_function(amplitudes, means, standard_deviations)
kernel = make_kernel(kernel_radius)

# the growth function
mean_g = 0.26
standard_deviation_g = 0.036

clipping_fn = lambda x: np.clip(x,0,1.0)
my_update = make_update_function(mean_g, standard_deviation_g)


# The Platonic Pattern: _Hydrogeminium natans_ pickle

In [ ]:
pattern_filepath = os.path.join("..", "patterns", "hydrogeminium_natans_pickle.npy")

pattern = np.load(pattern_filepath)[None,None,:,:]
plt.figure()
plt.imshow(pattern.squeeze(), cmap="magma")


In [ ]:
number_samples = 1
dim = 256
grid = np.zeros((number_samples,1,dim,dim))
grid = grid.at[:,:,:pattern.shape[-2], :pattern.shape[-1]].set(pattern)

dts = 0.1

clipping_fn = lambda x: np.clip(x,0,1.0)
update_step = make_update_step(my_update, kernel, dts, clipping_fn, decimals=32)

num_frames = 100

fig, ax = get_fig(grid[0])
plt.show()

IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_frame, frames=num_frames, interval=10).to_jshtml())

In [ ]:
min_dt = 0.001
max_dt = 0.2
number_dt_steps = 4
min_kr = 5
max_kr = 63
number_kr_steps = 4

dts = np.arange(min_dt, max_dt, (max_dt-min_dt) / number_dt_steps)[:,None,None,None]
krs = np.arange(min_kr, max_kr, (max_kr-min_kr) / number_kr_steps)
max_t = dts.max() * 100

number_samples = 1
dim = 256
grid = np.zeros((number_dt_steps,number_kr_steps,dim,dim))
grid = grid.at[:,:,:pattern.shape[-2], :pattern.shape[-1]].set(pattern)

kernel_dim = 124
make_kernel = make_make_kernel_function(amplitudes, means, standard_deviations, dim=kernel_dim)

kernels = np.zeros((1, number_kr_steps, kernel_dim+1, kernel_dim+1))

for ii in range(number_kr_steps):
    kernels = kernels.at[:,ii:ii+1,:,:].set(make_kernel(krs[ii]))

if(0):
    plt.imshow(kernel.squeeze())
    plt.show()

update_step = make_update_step(my_update, kernel, dts, clipping_fn)

for step in range(100):
    grid = update_step(grid)

plt.figure()
plt.subplot(221)
plt.imshow(grid[0,0])
plt.subplot(222)
plt.imshow(grid[1,0])
plt.subplot(223)
plt.imshow(grid[2,0])
plt.subplot(224)
plt.imshow(grid[3,0])
plt.show()


plt.figure()
plt.subplot(221)
plt.imshow(kernels[0,0])
plt.subplot(222)
plt.imshow(kernels[0,1])
plt.subplot(223)
plt.imshow(kernels[0,2])
plt.subplot(224)
plt.imshow(kernels[0,3])
plt.show()

kernel.shape

In [ ]:
min_dt = 0.001
max_dt = 0.2
number_dt_steps = 4
min_kr = 25
max_kr = 33
number_kr_steps = 4

for number_steps in [4]:
    number_dt_steps = 512
    number_kr_steps = 4
    dts = np.arange(min_dt, max_dt, (max_dt-min_dt) / number_dt_steps)[:,None,None,None]
    krs = np.arange(min_kr, max_kr, (max_kr-min_kr) / number_kr_steps)
    max_t = dts.max() * 100
    
    number_samples = 1
    dim = 128
    grid = np.zeros((number_dt_steps,number_kr_steps,dim,dim))
    grid = grid.at[:,:,:pattern.shape[-2], :pattern.shape[-1]].set(pattern)
    
    kernel_dim = 125
    make_kernel = make_make_kernel_function(amplitudes, means, standard_deviations, dim=kernel_dim)
    
    kernels = np.zeros((1, number_kr_steps, kernel_dim+1, kernel_dim+1))
    
    for ii in range(number_kr_steps):
        kernels = kernels.at[:,ii:ii+1,:,:].set(make_kernel(krs[ii]))

    
    update_step = make_update_step(my_update, kernels, dts, clipping_fn)

    t0 = time.time()
    for step in range(1000):
        grid = update_step(grid)
        
    t1 = time.time()

    print(f"time elapsed = {t1-t0:.3f} for ndt x nkr = {dts.shape[0]} x {kernels.shape[1]}")
    print(f" \t {1000 * (dts.shape[0] *kernels.shape[1]) / (t1-t0):.4f} grid updates/second")

In [ ]:
results

In [ ]:
time elapsed = 28.087 for ndt x nkr = 1024 x 4
 	 145830.8121 grid updates/second
time elapsed = 107.733 for ndt x nkr = 4096 x 4
 	 152079.7693 grid updates/second

In [ ]:
time elapsed = 24.300 for ndt x nkr = 512 x 1
 	 21069.7054 grid updates/second


time elapsed = 184.229 for ndt x nkr = 4096 x 1
 	 22233.1554 grid updates/second
time elapsed = 187.316 for ndt x nkr = 1024 x 4
 	 21866.7657 grid updates/second

time elapsed = 126.403 for ndt x nkr = 1 x 2048
 	 16202.1797 grid updates/second
time elapsed = 254.049 for ndt x nkr = 1 x 4096
 	 16122.8987 grid updates/second
time elapsed = 47.824 for ndt x nkr = 1024 x 1
 	 21411.7405 grid updates/second
time elapsed = 63.424 for ndt x nkr = 1 x 1024
 	 16145.2596 grid updates/second


time elapsed = 3.853 for ndt x nkr = 4 x 4
 	 4152.5221 grid updates/second
time elapsed = 4.454 for ndt x nkr = 8 x 8
 	 14368.4065 grid updates/second
time elapsed = 48.361 for ndt x nkr = 32 x 32
 	 21174.1202 grid updates/second
time elapsed = 106.580 for ndt x nkr = 48 x 48
 	 21617.6270 grid updates/second
time elapsed = 188.560 for ndt x nkr = 64 x 64
 	 21722.5312 grid updates/second

In [ ]:

grid = np.zeros((number_dt_steps,number_kr_steps,dim,dim))
grid[:,:,:pattern.shape[-2], :pattern.shape[-1]] = pattern

nbhd = ft_convolve(grid, kernels)

nbhd.shape


plt.figure()
for jj in range(4):
    plt.subplot(2,2,jj+1)
    plt.imshow(nbhd[0,jj])


plt.figure()
for kk in range(4):
    plt.subplot(2,2,kk+1)
    plt.imshow(nbhd[kk,1])
plt.show()

In [ ]:
grid.shape

In [ ]:

plt.figure()
plt.subplot(221)
plt.imshow(grid[0,0])
plt.subplot(222)
plt.imshow(grid[1,0])
plt.subplot(223)
plt.imshow(grid[2,0])
plt.subplot(224)
plt.imshow(grid[3,0])
plt.show()


plt.figure()
plt.subplot(221)
plt.imshow(grid[2,0])
plt.subplot(222)
plt.imshow(grid[2,1])
plt.subplot(223)
plt.imshow(grid[2,2])
plt.subplot(224)
plt.imshow(grid[2,3])
plt.show()

plt.figure()
plt.subplot(221)
plt.imshow(kernels[0,0])
plt.subplot(222)
plt.imshow(kernels[0,1])
plt.subplot(223)
plt.imshow(kernels[0,2])
plt.subplot(224)
plt.imshow(kernels[0,3])
plt.show()

In [ ]:
min_dt = 0.001
max_dt = 1.0
number_dt_steps = 16
min_kr = 5
max_kr = 63
number_kr_steps = 16

dts = np.arange(min_dt, max_dt, (max_dt-min_dt) / number_dt_steps)
krs = np.arange(min_kr, max_kr, (max_kr-min_kr) / number_kr_steps)
max_t = dts.max() * 100

number_samples = 1
dim = 256
grid = np.zeros((number_samples,1,dim,dim))

make_kernel = make_make_kernel_function(amplitudes, means, standard_deviations, dim=190)
t0 = time.time()

results = stability_sweep(dts, krs, grid, pattern, make_kernel, my_update, max_t, max_steps=100)
t1 = time.time()

print(f"elapsed: {t1-t0:.4f}")

fig, ax = plt.subplots(1,1, figsize=(20,20))
ax.imshow(results)
_ = ax.set_yticks(np.arange(0,dts.shape[0]))
_ = ax.set_yticklabels([f"{elem:.4f}" for elem in dts])
_ = ax.set_xticks(np.arange(0,krs.shape[0]))
_ = ax.set_xticklabels([f"{elem:.4f}" for elem in krs], rotation=90)

# The Non-Platonic _H. natans_ wobbler

In [ ]:
#
pattern_filepath = os.path.join("..", "patterns", "hydrogeminium_natans_wobbler.npy")

pattern = np.load(pattern_filepath)[None,None,:,:]
plt.figure()
plt.imshow(pattern.squeeze(), cmap="magma")


In [ ]:
number_samples = 1
dim = 256
grid = np.zeros((number_samples,1,dim,dim))
grid[:,:,:pattern.shape[-2], :pattern.shape[-1]] = pattern

dts = 0.38

clipping_fn = lambda x: np.clip(x,0,1.0)
update_step = make_update_step(my_update, kernel, dts, clipping_fn, decimals=32)

num_frames = 100

fig, ax = get_fig(grid[0])
plt.show()

IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_frame, frames=num_frames, interval=100).to_jshtml())

In [ ]:
min_dt = 0.001
max_dt = 1.0
number_dt_steps = 16
min_kr = 5
max_kr = 63
number_kr_steps = 16

dts = np.arange(min_dt, max_dt, (max_dt-min_dt) / number_dt_steps)
krs = np.arange(min_kr, max_kr, (max_kr-min_kr) / number_kr_steps)
max_t = dts.max() * 100

number_samples = 1
dim = 256
grid = np.zeros((number_samples,1,dim,dim))

make_kernel = make_make_kernel_function(amplitudes, means, standard_deviations, dim=190)
t0 = time.time()

results = stability_sweep(dts, krs, grid, pattern, make_kernel, my_update, max_t, max_steps=100)
t1 = time.time()

print(f"elapsed: {t1-t0:.4f}")

fig, ax = plt.subplots(1,1, figsize=(20,20))
ax.imshow(results)
_ = ax.set_yticks(np.arange(0,dts.shape[0]))
_ = ax.set_yticklabels([f"{elem:.4f}" for elem in dts])
_ = ax.set_xticks(np.arange(0,krs.shape[0]))
_ = ax.set_xticklabels([f"{elem:.4f}" for elem in krs], rotation=90)

In [ ]:
min_dt = 0.43
max_dt = 0.46
number_dt_steps = 16
min_kr = 43
max_kr = 46
number_kr_steps = 16

dts = np.arange(min_dt, max_dt, (max_dt-min_dt) / number_dt_steps)
krs = np.arange(min_kr, max_kr, (max_kr-min_kr) / number_kr_steps)
max_t = dts.max() * 100

number_samples = 1
dim = 256
grid = np.zeros((number_samples,1,dim,dim))

make_kernel = make_make_kernel_function(amplitudes, means, standard_deviations, dim=190)
t0 = time.time()

results = stability_sweep(dts, krs, grid, pattern, make_kernel, my_update, max_t, max_steps=100)
t1 = time.time()

print(f"elapsed: {t1-t0:.4f}")

fig, ax = plt.subplots(1,1, figsize=(20,20))
ax.imshow(results)
_ = ax.set_yticks(np.arange(0,dts.shape[0]))
_ = ax.set_yticklabels([f"{elem:.4f}" for elem in dts])
_ = ax.set_xticks(np.arange(0,krs.shape[0]))
_ = ax.set_xticklabels([f"{elem:.4f}" for elem in krs], rotation=90)